In [2]:
import requests
import json
import os
from dotenv import load_dotenv

api_key = os.getenv("TVFY_API_KEY")

In [3]:
def fetch_additional_info(member_id, api_key):
    url = f"https://theyvoteforyou.org.au/api/v1/people/{member_id}.json?key={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

In [5]:
result = fetch_additional_info(10904, api_key)
print(result.keys())

dict_keys(['id', 'latest_member', 'rebellions', 'votes_attended', 'votes_possible', 'offices', 'policy_comparisons'])


In [9]:
# Mapping dictionary for 'Effective Party' column.
effective_party_map = {
    'Country Liberal Party': 'Liberal National Party',
    'DPRES': 'Australian Labor Party',
    'Liberal Party': 'Liberal National Party',
    'National Party': 'Liberal National Party',
    'PRES': 'Australian Labor Party',
    'Centre Alliance': 'Independent',
    'CWM': 'Liberal National Party',
    'Katter\'s Australian Party': 'Independent',
    'SPK': 'Australian Labor Party'
}


def flatten_member_info(member):
    latest_member = member['latest_member']
    party = latest_member['party']
    effective_party = effective_party_map.get(party, party)  # Get effective party from map, or use original party if not found

    additional_info = member.get('additional_info', {})

    return {
        'id': member['id'],
        'name': f"{latest_member['name']['first']} {latest_member['name']['last']}",
        'electorate': latest_member['electorate'],
        'house': latest_member['house'],
        'party': latest_member['party'],
        'effective_party': effective_party,
        'rebellions': additional_info.get('rebellions'),
        'votes_attended': additional_info.get('votes_attended'),
        'votes_possible': additional_info.get('votes_possible'),
        'offices': [office['position'] for office in additional_info.get('offices', [])]
    }


In [10]:
flattened_member = flatten_member_info(result)
print(flattened_member)

{'id': 10904, 'name': 'Jim Molan', 'electorate': 'NSW', 'house': 'senate', 'party': 'Liberal Party', 'effective_party': 'Liberal National Party', 'rebellions': None, 'votes_attended': None, 'votes_possible': None, 'offices': []}


In [12]:
def load_members_from_files(filename):
    # Check if the file with all members exists
    if os.path.exists(filename):
        # Load data from the file if it exists
        with open(filename, 'r') as f:
            return json.load(f)
    else:
        print("No local member data found.")
        return None

In [16]:
# OS setup
senate_filename = "../data/parliament/senate.json"
current_senators = load_members_from_files(senate_filename)

print(current_senators[0].keys())


dict_keys(['id', 'name', 'electorate', 'house', 'party', 'effective_party', 'rebellions', 'votes_attended', 'votes_possible', 'offices'])


In [19]:
print(len(current_senators))

77


In [18]:
current_senators.append(flattened_member)

In [ ]:
print(len(current_senators))

In [21]:
with open(senate_filename, 'w') as f:
    json.dump(current_senators, f)